In [1]:
import scipy
import sys
from lxml import etree
import FeatureExtraction as FE
sys.path.insert(0, '../notebooks/libs/')


In [2]:
#Jedna linija je jedan dokument.
#Lista svih linija.
#id - autor, broj njegove linije
pathToFile='../../dataset/training/pan12-sexual-predator-identification-training-corpus-2012-05-01.xml'
pathToIds = '../../dataset/training/pan12-sexual-predator-identification-training-corpus-predators-2012-05-01.txt'
tree=etree.parse(pathToFile)
predators_ids = FE.sexual_predator_ids(pathToIds)

In [ ]:
predators_ids

In [ ]:
def prepare_for_tf_idf(path_to_dataset_xml):
    tree=etree.parse(path_to_dataset_xml)
    message_node = extract_message_nodes_as_list_from_parsed_text(tree)
    dictionary= extract_author_text_dictionary_from_message_nodes(message_node)
    filter_words_from_dictionary(dictionary) 
    list_of_authors_strings = []
    for key in sorted(dictionary):
        tmp = dictionary.get(key)
        if None in tmp:
            dictionary[key]=''
        list_of_authors_strings.append(' '.join(dictionary.get(key)))
    return list_of_authors_strings

In [3]:
message_nodes = FE.extract_message_nodes_as_list_from_parsed_text(tree)

message_node?

In [19]:
message_node = message_nodes[100]
lista = message_node.xpath('./author')
for l in lista:
    print str(l.text)

c18a32dc7dc8b612f47c3d21f6ccf5f0


In [ ]:
text_nodes=[]
for message_node in message_nodes:
    text_node = message_node.xpath('./text')
    for node in text_node:
        text_nodes.append(node.text)        

In [ ]:
text_nodes?

In [23]:
def extract_all_chat_lines_from_XML(pathToFile,predators_ids):
    tree=etree.parse(pathToFile)
    message_nodes = FE.extract_message_nodes_as_list_from_parsed_text(tree)
    text_nodes=[]
    for message_node in message_nodes:
        list_of_authors = message_node.xpath('./author')
        for author in list_of_authors:
            if str(author.text) in predators_ids:
                text_node = message_node.xpath('./text')
            else:
                continue
            for node in text_node:
                if node.text is None:
                    continue
                text_nodes.append(node.text)
    return text_nodes

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
def make_tf_idf_matrix_for_clustering_from_XML(pathToFile, predators_ids):
    text_nodes=extract_all_chat_lines_from_XML(pathToFile,predators_ids)
    tfidf=TfidfVectorizer(stop_words='english',min_df=10)
    #X = tfidf.fit_transform(list_of_authors_strings)
    #idf = tfidf._tfidf
    return tfidf.fit_transform(text_nodes), tfidf.get_feature_names()
    #return dict(zip(tfidf.get_feature_names(), idf))
    #return tfidf.get_feature_names(),idf

In [25]:
matrix,lines= make_tf_idf_matrix_for_clustering_from_XML('../../dataset/training/pan12-sexual-predator-identification-training-corpus-2012-05-01.xml',predators_ids)

In [30]:
matrix[0]

<1x977 sparse matrix of type '<type 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [27]:
from sklearn.cluster import KMeans

In [31]:
random_state = 170
y_pred = KMeans(n_clusters=2, random_state=random_state).fit_predict(matrix)

In [29]:
y_pred?